### GENERATE TRAJECTORIES USING **DOUBLE-WELL POTENTIAL**

using different physical parameters (Stefano's)

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
#from scipy.constants import Boltzmann as kB
from numpy.random import randn as gauss
from numpy.random import rand as uniform


kB = 1
### Physical parameters 
R = 1e-7                                # Radius of the Brownian particle [m]
eta = 0.001                             # Viscosity of the medium [kg m^-1 s^-1]
T = 1                                 # Temperature [K]
L0 = 5                               # Reference distance from middle to one minimum [m]
H0 = 4                             # Barrier height [Joule]
gamma0 = 1            # Reference friction coefficient [kg s^-1]

### Simulation parameters
N = 1000                   # Number of samples of the trajectory
Dt = 1                  # Timestep 
oversampling = 5           # Simulation oversampling
offset = 1000              # Number of equilibration timesteps
batch_size = 32            # Number of trajectories

### Define functions to scale and rescale inputs
scale_inputs = lambda x: x * 1e+6                    # Scales input trajectory to order 1
rescale_inputs = lambda scaled_x: scaled_x * 1e-6    # Rescales input trajectory to physical units

### Define function to scale and rescale targets
scale_targets = lambda L, H: [L/L0 -1, np.log(H / H0)]                        # Scales targets to order 1
rescale_targets = lambda scaled_L, scaled_H: [(1 + scaled_L)*L0*1e6, 
                                              np.exp(scaled_H) * H0/kB/300] # Inverse of targets_scaling

def simulate_trajectory(batch_size=batch_size, 
                        T=T,
                        H0=H0,
                        L0=L0,
                        gamma0=gamma0,
                        N=N, 
                        Dt=Dt, 
                        oversampling=oversampling, 
                        offset=offset):#, 
                        #scale_inputs=scale_inputs, 
                        #scale_targets=scale_targets):
    
    ### Randomize trajectory parameters
    L = L0 * (uniform(batch_size)+.5) 
    H = H0 * 10**(uniform(batch_size)*1.75 - .75)       # Generates random values for computing the stiffness
    gamma = gamma0 * (uniform(batch_size) * .1 + .95)   # Marginal randomization of friction coefficient to tolarate small changes

    ### Simulate
    dt = Dt / oversampling                 # time step of the simulation
    x = np.zeros((batch_size, N))          # initialization of the x array
    k0 = 4*H/L**2 
    k1 = 4*H/L**4
    D = kB * T / gamma                     # diffusion coefficient
    C1 = +k0 / gamma * dt
    C2 = -k1 / gamma * dt                  # drift coefficient of the Langevin equation
    C3 = np.sqrt(2 * D * dt)               # random walk coefficient of the Langevin equation
    X = x[:, 0]
    n = 0
    
    for t in range(offset):                      # Offset (for some prerun before running)
        X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)
        #X = X - (2 * X**3) + (12 * X**2) - (18 * X) + 3 + (C3 * gauss(batch_size))
        
    for t in range(N * oversampling):            # Simulation                
        X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)
        #X = X - (2 * X**3) + (12 * X**2) - (18 * X) + 3 + (C3 * gauss(batch_size))
        if t % oversampling == 0:                # We save every oversampling^th values 
            x[:, n] = X 
            n += 1

    inputs = scale_inputs(x)
    targets = np.swapaxes(scale_targets(*[L, H]),0,1)
    target_reals = np.swapaxes([L*1e6, H/kB/300],0,1)

    return inputs, targets, target_reals

##### TRAIN THE DEEP NEURAL NETWORK

In [2]:
def train_deep_learning_network(
    network,
    simulate_trajectory,
    sample_sizes = (32, 128, 512),#(32, 128, 512, 2048),
    iteration_numbers = (3001, 2001, 1001),#(1001, 2001, 3001),#(3001, 2001, 1001, 101),
    verbose=.1):
    """Train a deep learning network.
    
    Input:
    network: deep learning network
    simulate_trajectory: trajectory generator function
    sample_sizes: sizes of the batches of trajectories used in the training [tuple of positive integers]
    iteration_numbers: numbers of batches used in the training [tuple of positive integers]
    verbose: frequency of the update messages [number between 0 and 1]
        
    Output:
    training_history: dictionary with training history
    """  
    
    import numpy as np
    from time import time
     
    training_history = {}
    training_history['Sample Size'] = []
    training_history['Iteration Number'] = []
    training_history['Iteration Time'] = []
    training_history['MSE'] = []
    training_history['MAE'] = []
    
    for sample_size, iteration_number in zip(sample_sizes, iteration_numbers):
        for iteration in range(iteration_number):
            
            # measure initial time for iteration
            initial_time = time()

            # generate trajectories and targets
            network_blocksize = network.get_layer(index=0).get_config()['batch_input_shape'][1:][1]                        
            number_of_outputs = network.get_layer(index=-1).get_config()['units']
            output_shape = (sample_size, number_of_outputs)
            targets = np.zeros(output_shape)
            
            
            batch_size = sample_size
            trajectory, target, target_real = simulate_trajectory(batch_size)
            #trajectory = trajectory.scaled_values
            trajectory_dimensions = [sample_size, round(trajectory.size/network_blocksize/sample_size), network_blocksize]
            trajectories = np.array(trajectory).reshape(trajectory_dimensions)
            targets = target#.scaled_values
                
                

            # training
            history = network.fit(trajectories,
                                targets,
                                epochs=1, 
                                batch_size=sample_size,
                                verbose=False)
                        
            # measure elapsed time during iteration
            iteration_time = time() - initial_time

            # record training history
            mse = history.history['mse'][0]
            mae = history.history['mae'][0]
                        
            training_history['Sample Size'].append(sample_size)
            training_history['Iteration Number'].append(iteration)
            training_history['Iteration Time'].append(iteration_time)
            training_history['MSE'].append(mse)
            training_history['MAE'].append(mae)

            if not(iteration%int(verbose**-1)):
                print('Sample size %6d   iteration number %6d   MSE %10.4f   MAE %10.4f   Time %10f ms' % (sample_size, iteration + 1, mse, mae, iteration_time * 1000))
                
    return training_history

##### PLOT THE PERFORMANCE

In [3]:
def plot_learning_performance(training_history, number_of_timesteps_for_average = 100, figsize=(20,20)):
    """Plot the learning performance of the deep learning network.
    
    Input:
    training_history: dictionary with training history, typically obtained from train_deep_learning_network()
    number_of_timesteps_for_average: length of the average [positive integer number]
    figsize: figure size [list of two positive numbers]
        
    Output: none
    """    

    import matplotlib.pyplot as plt
    from numpy import convolve, ones
    
    plt.figure(figsize=figsize)

    plt.subplot(5, 1, 1)
    plt.semilogy(training_history['MSE'], 'k')
    plt.semilogy(convolve(training_history['MSE'], ones(number_of_timesteps_for_average) / number_of_timesteps_for_average, mode='valid'), 'r')
    plt.ylabel('MSE', fontsize=24)
    plt.xlabel('Epochs', fontsize=24)

    plt.subplot(5, 1, 2)
    plt.semilogy(training_history['MAE'], 'k')
    plt.semilogy(convolve(training_history['MAE'], ones(number_of_timesteps_for_average) / number_of_timesteps_for_average, mode='valid'), 'r')
    plt.ylabel('MAE', fontsize=24)
    plt.xlabel('Epochs', fontsize=24)
    plt.show()

##### TEST THE DNN ON NEW SIMULATED TRAJECTORIES

In [4]:
def predict(network, trajectory):
    """ Predict parameters of the force field from the trajectory using the deep learnign network.
    
    Inputs:
    network: deep learning network
    image: trajectroy [numpy array of real numbers]
    
    Output:
    predicted_targets: predicted parameters of the calibrated force field [1D numpy array containing outputs]
    """
    
    from numpy import reshape
    
    network_blocksize = network.get_layer(index=0).get_config()['batch_input_shape'][1:][1]
    predicted_targets = network.predict(reshape(trajectory, [1,round(trajectory.size/network_blocksize),network_blocksize]))   
        
    return predicted_targets


def test_performance(simulate_trajectory, network, rescale_targets, number_of_predictions_to_show=100):#, dt = 1e-1):

    
    network_blocksize = network.get_layer(index=0).get_config()['batch_input_shape'][1:][1]


    predictions_scaled = []
    predictions_physical = []

    batch_size = number_of_predictions_to_show
    trajectory, targets, targets_real = simulate_trajectory(batch_size)
    targets_physical = list(targets_real)#targets.values)
    targets_scaled = list(targets)#.scaled_values)
    #trajectory = trajectory.scaled_values
    trajectory_dimensions = [number_of_predictions_to_show, round(trajectory.size/network_blocksize/number_of_predictions_to_show) , network_blocksize]
    trajectories = np.array(trajectory).reshape(trajectory_dimensions)
       

    for i in range(number_of_predictions_to_show):
        predictions = predict(network, trajectories[i])


        predictions_scaled.append(predictions[0])
        predictions_physical.append(rescale_targets(*predictions[0]))

    number_of_outputs = network.get_layer(index=-1).get_config()['units']    

    targets_physical = np.array(targets_physical).transpose()
    targets_scaled = np.array(targets_scaled).transpose()
    predictions_scaled = np.array(predictions_scaled).transpose()
    predictions_physical = np.array(predictions_physical).transpose()

    # Do not show results at the edges of the training range 

    if number_of_outputs>1:

        ind = np.isfinite(targets_scaled[0])
        for target_number in range(number_of_outputs):
            target_max = .9 * np.max(targets_scaled[target_number]) + .1 * np.min(targets_scaled[target_number])
            target_min = .1 * np.max(targets_scaled[target_number]) + .9 * np.min(targets_scaled[target_number])
            ind = np.logical_and(ind, targets_scaled[target_number] < target_max)
            ind = np.logical_and(ind, targets_scaled[target_number] > target_min)
    else:
        target_max = .9 * np.max(targets_scaled) + .1 * np.min(targets_scaled)
        target_min = .1 * np.max(targets_scaled) + .9 * np.min(targets_scaled)
        ind = np.logical_and(targets_scaled < target_max, targets_scaled > target_min)

    return targets_scaled, targets_physical, predictions_scaled, predictions_physical


def plot_test_performance(targets_scaled, targets_physical, predictions_scaled, predictions_physical, network):
    
    import matplotlib.pyplot as plt
    import numpy as np
    #from . import predict

    number_of_outputs = network.get_layer(index=-1).get_config()['units']    
    
    if number_of_outputs>1:

        for target_number in range(number_of_outputs):
            plt.figure(figsize=(20, 10))

            plt.subplot(121)
            plt.plot(targets_scaled[target_number],
                     predictions_scaled[target_number],
                     '.')
            #plt.xlabel(targets.scalings[target_number], fontsize=18)
            #plt.ylabel('Predicted ' + targets.scalings[target_number], fontsize=18)
            plt.axis('square')
            plt.title('Prediction performance in scaled units', fontsize=18)

            plt.subplot(122)
            plt.plot(targets_physical[target_number],
                     predictions_physical[target_number],
                    '.')
            #plt.xlabel(targets.names[target_number], fontsize=18)
            #plt.ylabel('Predicted ' + targets.names[target_number], fontsize=18)
            plt.axis('square')
            plt.title('Prediction performance in real units', fontsize=18)


    else: 
        plt.figure(figsize=(20, 10))

        plt.subplot(121)
        plt.plot(targets_scaled[ind],
                 predictions_scaled.transpose()[ind],
                 '.')
        #plt.xlabel(targets.scalings[0], fontsize=18)
        #plt.ylabel('Predicted ' + targets.scalings[0], fontsize=18)
        plt.axis('square')
        plt.title('Prediction performance in scaled units', fontsize=18)

        plt.subplot(122)
        plt.plot(targets_physical[ind],
                 predictions_physical.transpose()[ind],
                '.')
        #plt.xlabel(targets.names[0], fontsize=18)
        #plt.ylabel('Predicted ' + targets.names[0], fontsize=18)
        plt.axis('square')
        plt.title('Prediction performance in real units', fontsize=18)

## LSTM

CREATE MODEL

In [5]:
def create_deep_learning_network_LSTM(
    input_shape=(None, 50),
    lstm_layers_dimensions=(100, 50),
    number_of_outputs=2) :
    """Creates and compiles a deep learning network.
    
    Inputs:    
    input_shape: Should be the same size of the input trajectory []
    lstm_layers_dimensions: number of neurons in each LSTM layer [tuple of positive integers]
        
    Output:
    network: deep learning network
    """    

    from keras import models, layers, optimizers
    
    ### INITIALIZE DEEP LEARNING NETWORK
    network = models.Sequential()

    ### CONVOLUTIONAL BASIS
    for lstm_layer_number, lstm_layer_dimension in zip(range(len(lstm_layers_dimensions)), lstm_layers_dimensions):

        # add LSTM layer
        lstm_layer_name = 'lstm_' + str(lstm_layer_number + 1)
        if lstm_layer_number + 1 < len(lstm_layers_dimensions): # All layers but last
            lstm_layer = layers.LSTM(lstm_layer_dimension,
                                     return_sequences=True,
                                     dropout=0,
                                     recurrent_dropout=0,
                                     input_shape=input_shape,
                                     name=lstm_layer_name)
        else: # Last layer
            lstm_layer = layers.LSTM(lstm_layer_dimension,
                                     return_sequences=False,
                                     dropout=0,
                                     recurrent_dropout=0,
                                     input_shape=input_shape,
                                     name=lstm_layer_name)

        network.add(lstm_layer)
    # OUTPUT LAYER
    output_layer = layers.Dense(number_of_outputs, name='output')
    network.add(output_layer)
    
    network.compile(optimizer=optimizers.Adam(lr=1e-3), loss='mse', metrics=['mse', 'mae'])
    
    return network

In [6]:
### Create deep learning network
networkLSTM = create_deep_learning_network_LSTM()

### Print deep learning network summary
networkLSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, None, 100)         60400     
                                                                 
 lstm_2 (LSTM)               (None, 50)                30200     
                                                                 
 output (Dense)              (None, 2)                 102       
                                                                 
Total params: 90,702
Trainable params: 90,702
Non-trainable params: 0
_________________________________________________________________


C:\Users\fap_9\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


#### TRAIN MODEL

In [7]:
%%time

training_history_LSTM = train_deep_learning_network(networkLSTM, simulate_trajectory)

C:\Users\fap_9\AppData\Local\Temp\ipykernel_28276\2641657375.py:64: RuntimeWarning: overflow encountered in power
  X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)
C:\Users\fap_9\AppData\Local\Temp\ipykernel_28276\2641657375.py:64: RuntimeWarning: invalid value encountered in add
  X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)
C:\Users\fap_9\AppData\Local\Temp\ipykernel_28276\2641657375.py:68: RuntimeWarning: overflow encountered in power
  X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)
C:\Users\fap_9\AppData\Local\Temp\ipykernel_28276\2641657375.py:68: RuntimeWarning: invalid value encountered in add
  X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)


Sample size     32   iteration number      1   MSE        nan   MAE        nan   Time 3712.594748 ms


C:\Users\fap_9\anaconda3\lib\site-packages\tensorflow\python\framework\constant_op.py:102: RuntimeWarning: overflow encountered in cast
  return ops.EagerTensor(value, ctx.device_name, dtype)


Sample size     32   iteration number     11   MSE        nan   MAE        nan   Time 175.868988 ms
Sample size     32   iteration number     21   MSE        nan   MAE        nan   Time 150.368214 ms
Sample size     32   iteration number     31   MSE        nan   MAE        nan   Time 139.695168 ms
Sample size     32   iteration number     41   MSE        nan   MAE        nan   Time 135.609388 ms
Sample size     32   iteration number     51   MSE        nan   MAE        nan   Time 162.472248 ms
Sample size     32   iteration number     61   MSE        nan   MAE        nan   Time 155.932903 ms
Sample size     32   iteration number     71   MSE        nan   MAE        nan   Time 174.909115 ms
Sample size     32   iteration number     81   MSE        nan   MAE        nan   Time 160.486937 ms
Sample size     32   iteration number     91   MSE        nan   MAE        nan   Time 191.941023 ms
Sample size     32   iteration number    101   MSE        nan   MAE        nan   Time 147.117376 ms


Sample size     32   iteration number    831   MSE        nan   MAE        nan   Time 176.726818 ms
Sample size     32   iteration number    841   MSE        nan   MAE        nan   Time 167.936325 ms
Sample size     32   iteration number    851   MSE        nan   MAE        nan   Time 170.951366 ms
Sample size     32   iteration number    861   MSE        nan   MAE        nan   Time 148.912907 ms
Sample size     32   iteration number    871   MSE        nan   MAE        nan   Time 164.078474 ms
Sample size     32   iteration number    881   MSE        nan   MAE        nan   Time 163.625956 ms
Sample size     32   iteration number    891   MSE        nan   MAE        nan   Time 180.393934 ms
Sample size     32   iteration number    901   MSE        nan   MAE        nan   Time 180.077076 ms
Sample size     32   iteration number    911   MSE        nan   MAE        nan   Time 172.222614 ms
Sample size     32   iteration number    921   MSE        nan   MAE        nan   Time 179.258108 ms


...

Sample size    128   iteration number    311   MSE        nan   MAE        nan   Time 590.553045 ms
Sample size    128   iteration number    321   MSE        nan   MAE        nan   Time 704.616308 ms
Sample size    128   iteration number    331   MSE        nan   MAE        nan   Time 644.319057 ms
Sample size    128   iteration number    341   MSE        nan   MAE        nan   Time 598.683119 ms
Sample size    128   iteration number    351   MSE        nan   MAE        nan   Time 593.576431 ms
Sample size    128   iteration number    361   MSE        nan   MAE        nan   Time 657.321453 ms
Sample size    128   iteration number    371   MSE        nan   MAE        nan   Time 738.103628 ms
Sample size    128   iteration number    381   MSE        nan   MAE        nan   Time 703.732967 ms
Sample size    128   iteration number    391   MSE        nan   MAE        nan   Time 710.250616 ms
Sample size    128   iteration number    401   MSE        nan   MAE        nan   Time 610.023260 ms


C:\Users\fap_9\AppData\Local\Temp\ipykernel_28276\2641657375.py:64: RuntimeWarning: overflow encountered in multiply
  X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)


Sample size    128   iteration number    591   MSE        nan   MAE        nan   Time 594.255447 ms
Sample size    128   iteration number    601   MSE        nan   MAE        nan   Time 611.154318 ms
Sample size    128   iteration number    611   MSE        nan   MAE        nan   Time 609.645844 ms
Sample size    128   iteration number    621   MSE        nan   MAE        nan   Time 812.320948 ms
Sample size    128   iteration number    631   MSE        nan   MAE        nan   Time 677.428961 ms
Sample size    128   iteration number    641   MSE        nan   MAE        nan   Time 665.608883 ms
Sample size    128   iteration number    651   MSE        nan   MAE        nan   Time 638.471603 ms
Sample size    128   iteration number    661   MSE        nan   MAE        nan   Time 609.735489 ms
Sample size    128   iteration number    671   MSE        nan   MAE        nan   Time 650.506973 ms
Sample size    128   iteration number    681   MSE        nan   MAE        nan   Time 544.239283 ms


Sample size    128   iteration number   1411   MSE        nan   MAE        nan   Time 702.547550 ms
Sample size    128   iteration number   1421   MSE        nan   MAE        nan   Time 557.817221 ms
Sample size    128   iteration number   1431   MSE        nan   MAE        nan   Time 588.615894 ms
Sample size    128   iteration number   1441   MSE        nan   MAE        nan   Time 646.784067 ms
Sample size    128   iteration number   1451   MSE        nan   MAE        nan   Time 639.055252 ms
Sample size    128   iteration number   1461   MSE        nan   MAE        nan   Time 822.823048 ms
Sample size    128   iteration number   1471   MSE        nan   MAE        nan   Time 681.426764 ms
Sample size    128   iteration number   1481   MSE        nan   MAE        nan   Time 531.666517 ms
Sample size    128   iteration number   1491   MSE        nan   MAE        nan   Time 1048.170090 ms
Sample size    128   iteration number   1501   MSE        nan   MAE        nan   Time 950.032711 ms

Sample size    512   iteration number    221   MSE        nan   MAE        nan   Time 1933.617353 ms
Sample size    512   iteration number    231   MSE        nan   MAE        nan   Time 1841.158867 ms
Sample size    512   iteration number    241   MSE        nan   MAE        nan   Time 1733.401299 ms
Sample size    512   iteration number    251   MSE        nan   MAE        nan   Time 2025.352478 ms
Sample size    512   iteration number    261   MSE        nan   MAE        nan   Time 1569.242954 ms
Sample size    512   iteration number    271   MSE        nan   MAE        nan   Time 1676.102161 ms
Sample size    512   iteration number    281   MSE        nan   MAE        nan   Time 1848.273516 ms
Sample size    512   iteration number    291   MSE        nan   MAE        nan   Time 1951.185703 ms
Sample size    512   iteration number    301   MSE        nan   MAE        nan   Time 1963.019133 ms
Sample size    512   iteration number    311   MSE        nan   MAE        nan   Time 1833.

#### TEST MODEL

In [8]:
%%time

number_of_predictions_to_show = 1000
prediction_LSTM_test = test_performance(simulate_trajectory, networkLSTM, rescale_targets, number_of_predictions_to_show)

C:\Users\fap_9\AppData\Local\Temp\ipykernel_28276\2641657375.py:64: RuntimeWarning: overflow encountered in power
  X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)
C:\Users\fap_9\AppData\Local\Temp\ipykernel_28276\2641657375.py:64: RuntimeWarning: invalid value encountered in add
  X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)
C:\Users\fap_9\AppData\Local\Temp\ipykernel_28276\2641657375.py:68: RuntimeWarning: overflow encountered in power
  X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)
C:\Users\fap_9\AppData\Local\Temp\ipykernel_28276\2641657375.py:68: RuntimeWarning: invalid value encountered in add
  X = X + C1 * X + C2 * X**3 + C3 * gauss(batch_size)


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step
CPU times: total: 46.8 s
Wall time: 1min 12s


#### Save data

In [ ]:
# save train data



### Plot results

In [ ]:
### Plot learning performance
number_of_timesteps_for_average = 100
plot_learning_performance(training_history_LSTM, number_of_timesteps_for_average)

In [ ]:
### Plot test performance
plot_test_performance(prediction_LSTM_test[0], prediction_LSTM_test[1], prediction_LSTM_test[2], prediction_LSTM_test[3], networkLSTM)